In [43]:
from os import path, getcwd
from IPython.display import Markdown as md
import pandas as pd
import numpy as np
import sqlalchemy as sa
import statsmodels.api as sm
import statsmodels.formula.api as smf
import matplotlib.pyplot as plt
import patsy

strYearDBPath=path.join('..','DB', 'year.sqlite3') # путь к рабочей базе данных SQLite Houses

# константы базы данных SQLite3
strYear_table='datas' # название таблицы в базе данных SQLite

#pd.set_option('display.max_rows',None)

In [212]:
strSELECT_Year='select * from {year_table}'.format(year_table=strYear_table)

conWork=sa.create_engine('sqlite+pysqlite:///{db_name}'.format(db_name=strYearDBPath)) # connection к рабочей базе данных

pdfYear=pd.read_sql(strSELECT_Year, con=conWork)

In [242]:
pdfNvstr=pdfYear[pdfYear['code']==134].set_index('date').drop(columns=['code'])
#pdfNvstr.rename(columns={'value': 'Запас жилья в строящихся МКД, на конец года (без учета строящихся за счет средств бюджетов)'},inplace=True)

Загрузка данных листа "Параметры" (27:26)

In [211]:
ddf=pd.read_excel(r'../DB/SRC/Модель развития ЖС_краткая версия_ИТОГ_Целевой_с новой моделью - новая доля продаж.xlsx',sheet_name=0,header=5,index_col=0)
ddf=ddf.loc[ddf.index.dropna()]
ddf=ddf.T

Загрузка данных листа "Жилищное строительство" (36:27)

In [206]:
hddf=pd.read_excel(r'../DB/SRC/Модель развития ЖС_краткая версия_ИТОГ_Целевой_с новой моделью - новая доля продаж.xlsx',sheet_name=3,header=5,index_col=0,nrows=45)
hddf=hddf.loc[hddf.index.dropna()]
hddf=hddf.iloc[:,1:-2]
#hddf.drop(hddf.columns[0],axis=1,inplace=True)
hddf=hddf.T

<div style="background:beige"><h3> Определение основных дат</h3>

  - `iLastFactYEAR` - последний год фактических данных. 2019
  - `iLastFORCAST` - год горизонта прогноза. 2030

    Пока задаются вручную
</div>

In [207]:
# iLastFactYEAR
iLastFactYEAR=conWork.connect().execute('select max(date) from datas').fetchone()[0]
iLastFORCAST=iLastFactYEAR+11

print('Последний фактический год - ', iLastFactYEAR, '; горизонт прогноза - ', iLastFORCAST)

Последний фактический год -  2019 ; горизонт прогноза -  2030


In [208]:
hddf.index=hddf.index.astype(int)
hddf_XY=hddf.truncate(after=iLastFactYEAR)
hddf_xy=hddf.truncate(before=iLastFactYEAR+1, after=iLastFORCAST)

In [210]:
hddf_XY

,Макроэкономические и демографические факторы,Темп прироста физического объема ВВП,"ИПЦ, в среднем за год",Уровень безработицы,Изменение реальных располагаемых доходов населения,"Численность населения, в среднем за год",Средний размер домохозяйства,Параметры данного блока,"Совокупный ввод жилья в рамках Национального проекта ""Жилье и городская среда""","Совокупный ввод жилья в МКД в рамках Национального проекта ""Жилье и городская среда""",...,Целевая плотность застройки в двадцати крупных городских агломерациях,Целевая плотность застройки в промышленных зонах городских центров двадцати крупных городских агломераций,Целевая плотность застройки на освоенных новых территориях в двадцати крупных городских агломерациях,Доля площади застраиваемых промышленных зон в черте двадцати центров крупных городских агломераций,Потенциал строительства жилья во всех промышленных зонах в черте двадцати центров крупных городских агломерациях,Коэффициент повышения плотности строительства в промышленных зонах,Параметры из других блоков модели,Средняя фактическая удельная стоимость строительства МКД,"Средняя цена 1 кв.м, приобретаемого по ДДУ, в МКД",Средняя цена 1 кв. м на первичном рынке
2005,NaN,6.37619,1.12706,0.0715324,1.124,143.519,2.66147,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13.812,21.1567,23.7716
2006,NaN,8.15343,1.09683,0.0706222,1.135,143.05,2.64589,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16.84,28.0115,31.4736
2007,NaN,8.53508,1.08988,0.0602939,1.121,142.805,2.6303,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20.72,39.056,43.8832
2008,NaN,5.25646,1.14105,0.0624119,1.024,142.742,2.61472,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,26.622,45.6864,51.333
2009,NaN,-7.82474,1.11698,0.0825224,1.03,142.785,2.59913,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,30.312,43.4902,48.8654
2010,NaN,18.3923,1.06851,0.0734454,1.059,142.849,2.58355,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,31.877,42.8293,48.1228
2011,NaN,0.467491,1.08463,0.0649564,1.005,142.961,2.54416,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,33.32,37.8975,42.5814
2012,NaN,4.0176,1.05067,0.0545866,1.046,143.202,2.50476,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,34.354,41.5987,46.7401
2013,NaN,1.75175,1.06761,0.0547804,1.04,143.507,2.46537,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,36.439,44.1202,49.5733
2014,NaN,0.744972,1.07804,0.0515632,0.988,146.091,2.42598,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,39.447,45.3205,50.9219


## Запас Жилья строящихся МКД

In [243]:
pdfNvstr['Тренд'] = None

In [256]:
pdfNvstr

,value,Тренд
date,,
2004,59.386684,63.125021
2005,66.863359,63.125021
2006,70.858939,68.861149
2007,68.941974,69.900456
2008,64.672504,66.807239
2009,68.476586,66.574545
2010,73.837691,71.157139
2011,81.098644,77.468168
2012,90.318934,85.708789


In [255]:
for i, row in pdfNvstr.iterrows():
    print(row.name)
    pdfNvstr['Тренд'][row.name+1]=np.mean([pdfNvstr['value'][row.name],pdfNvstr['value'][row.name+1]])

2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019


KeyError: 2020